In [1]:
import pandas
import numpy as np

In [2]:
item = dict()
mark = tuple()

In [3]:
df = pandas.read_csv("d_items.csv", engine="python", encoding="unicode_escape")
id_to_label = dict(zip(df["itemid"], df["label"]))
label_to_id = dict(zip(df["label"], df["itemid"]))
# df.head()

In [4]:
mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius", \
        "Temperature Fahrenheit")

In [18]:
%%time 
df = pandas.read_csv("chart_test3.csv", engine="python", encoding="unicode_escape")
# df.head()

CPU times: user 5.3 s, sys: 263 ms, total: 5.57 s
Wall time: 5.6 s


In [7]:
df['charttime'] = pandas.to_datetime(df['charttime']).round("10min")

In [6]:
# 建立一個CLASS容器來裝每一個PATIENT
class Patient:
    data = dict()
    def __init__(self, ):
        self.data = {"subject_id":None, "hadm_id":None, "charttime":None}
        for label_ in mark:
            self.data[label_] = None
    def allna(self, ):
        return not any([self.data[i]!=None for i in self.data])

In [9]:
output = []
tp = [label_to_id[i] for i in mark]
for name, group in df.groupby(["hadm_id", "charttime"]):
    patient = Patient()
    for id_, value_ in zip(group.itemid, group.value):
        if id_ in tp:
            patient.data[id_to_label[id_]] = value_
    # if not patient.allna():
    if True:
        patient.data["hadm_id"], patient.data["charttime"] = name
        if patient.data["Temperature Fahrenheit"] != None:
            patient.data["Temperature Celsius"] = round((float(patient.data["Temperature Fahrenheit"]) - 32) * 5 / 9, 1)
        output.append(patient.data)
merged_df = pandas.DataFrame(output,columns=['hadm_id','charttime']+list(mark)).drop(["Temperature Fahrenheit"], axis=1)
merged_df.to_csv("chart_test3_merge_new.csv", float_format='%.2f', index=0)

In [10]:
# df.to_csv("chart_test3_merge_new.csv", float_format='%.2f', index=0)
print("amount: ", len(merged_df))
# merged_df

amount:  101010


In [11]:
# ------------------------TODO------------------------ #

In [12]:
testdf = merged_df

In [13]:
testdf[1:10]

,hadm_id,charttime,Heart Rate,Respiratory Rate,Non Invasive Blood Pressure systolic,Non Invasive Blood Pressure diastolic,O2 saturation pulseoxymetry,Temperature Celsius
1,20005763,2175-05-31 02:50:00,96,15,122,66,96,37.1
2,20005763,2175-05-31 03:00:00,72,10,124,69,90,None
3,20005763,2175-05-31 04:00:00,87,12,118,58,95,36.7
4,20005763,2175-05-31 05:00:00,67,10,105,51,96,None
5,20005763,2175-05-31 06:00:00,67,12,95,68,94,None
6,20005763,2175-05-31 07:00:00,78,13,105,78,94,None
7,20005763,2175-05-31 08:00:00,70,9,105,69,97,None
8,20005763,2175-05-31 09:00:00,62,11,122,62,95,36.9
9,20005763,2175-05-31 09:50:00,None,None,None,None,None,None


In [14]:
new_mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius")
testdf[2:3][list(new_mark)]

,Heart Rate,Respiratory Rate,Non Invasive Blood Pressure systolic,Non Invasive Blood Pressure diastolic,O2 saturation pulseoxymetry,Temperature Celsius
2,72,10,124,69,90,None


In [15]:
df = pandas.read_csv("chart_test3_merge_new.csv", engine="python", encoding="unicode_escape")
t = 0
last_hadm_id = df.iloc[1].hadm_id
for i in range(1, len(df)):
    if last_hadm_id != df.iloc[i].hadm_id:
        df[t:i][list(new_mark)] = df[t:i][list(new_mark)].interpolate().fillna(method='ffill').fillna(method='bfill').fillna(value=df[t:i].mean())
        last_hadm_id = df.iloc[i].hadm_id
        t = i
# df[t:-1][list(new_mark)] = df[t:-1][list(new_mark)].interpolate().fillna(method='ffill').fillna(method='bfill').fillna(value=df[t:i].mean())
df[:-1].to_csv("chart_test3_e2.csv", float_format='%.1f', index=0)


/var/folders/bm/6f1rz86s3zg82c32tbrgx9j00000gn/T/ipykernel_6218/3897555827.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[t:i][list(new_mark)] = df[t:i][list(new_mark)].interpolate().fillna(method='ffill').fillna(method='bfill').fillna(value=df[t:i].mean())
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [16]:
import time

def read_csv_function(road: str) -> pandas.core.frame.DataFrame:
    return pandas.read_csv(road, engine='python')
def to_timestamp(s: str):
    try:
        return int(time.mktime(time.strptime(s, "%Y-%m-%d %H:%M:%S")))
    except ValueError:
        try:
            return int(time.mktime(time.strptime(s, "%Y-%m-%d %H:%M")))
        except ValueError:
            return int(time.mktime(time.strptime(s, "%Y/%m/%d %H:%M")))

print("READ procedureevents.csv")
df = read_csv_function("procedureevents.csv")
dic = {}
for i, it in enumerate(df.itemid):
    if it != 225466:
        continue
    if df.iloc[i].hadm_id not in dic:
        dic[df.iloc[i].hadm_id] = list()
    dic[df.iloc[i].hadm_id].append(to_timestamp(str(df.iloc[i].starttime)))

print("READ", "chart_test3_e2.csv")
df = read_csv_function("chart_test3_e2.csv")
ans = [False]*len(df)
for i in range(len(df)):
    if int(df.iloc[i].hadm_id) in dic:
        for j in dic[df.iloc[i].hadm_id]:
            if 0 <= j - to_timestamp(str(df.iloc[i].charttime)) <= 14400:
                ans[i] = True
                break

df["label"] = ans
df.to_csv("ANS.csv", float_format='%.2f', index=0)

READ procedureevents.csv
READ chart_test3_e2.csv
